# Population Density Correlation

In [2]:
import pandas as pd
from geopy.geocoders import GoogleV3

In [10]:
df_rent = pd.read_excel('../../ADS ASS2/Data/rent_2023.xlsx')

df_pode = pd.read_excel('../../ADS ASS2/Data/population_density_2023.xlsx')

In [11]:
df_rent

,Large Cities,Suburbs,2023-03-01 00:00:00,2023-06-01 00:00:00,2023-09-01 00:00:00,2023-12-01 00:00:00,Postcode
0,Inner Melbourne,Albert Park-Middle Park-West St Kilda,545,550,600,600,3182
1,NaN,Armadale,490,500,525,560,3143
2,NaN,Carlton North,620,630,650,670,3054
3,NaN,Carlton-Parkville,500,530,550,550,3053
4,NaN,CBD-St Kilda Rd,550,580,600,620,3000
...,...,...,...,...,...,...,...
137,NaN,Traralgon,385,390,390,395,3844
138,NaN,Wanagaratta,380,390,390,395,3677
139,NaN,Warragul,440,450,450,460,3820
140,NaN,Warrnambool,420,430,450,460,3280


In [12]:
df_rent.columns = ['Large Cities', 'Suburbs', '2023-03-01 00:00:00', '2023-06-01 00:00:00', '2023-09-01 00:00:00', '2023-12-01 00:00:00', 'Postcode']

df_rent['Average'] = df_rent[['2023-03-01 00:00:00', '2023-06-01 00:00:00', '2023-09-01 00:00:00', '2023-12-01 00:00:00']].mean(axis=1)

df_rent

,Large Cities,Suburbs,2023-03-01 00:00:00,2023-06-01 00:00:00,2023-09-01 00:00:00,2023-12-01 00:00:00,Postcode,Average
0,Inner Melbourne,Albert Park-Middle Park-West St Kilda,545,550,600,600,3182,573.75
1,NaN,Armadale,490,500,525,560,3143,518.75
2,NaN,Carlton North,620,630,650,670,3054,642.50
3,NaN,Carlton-Parkville,500,530,550,550,3053,532.50
4,NaN,CBD-St Kilda Rd,550,580,600,620,3000,587.50
...,...,...,...,...,...,...,...,...
137,NaN,Traralgon,385,390,390,395,3844,390.00
138,NaN,Wanagaratta,380,390,390,395,3677,388.75
139,NaN,Warragul,440,450,450,460,3820,450.00
140,NaN,Warrnambool,420,430,450,460,3280,440.00


In [13]:
df_subset_rent = df_rent[['Average', 'Postcode']]
df_subset_rent

,Average,Postcode
0,573.75,3182
1,518.75,3143
2,642.50,3054
3,532.50,3053
4,587.50,3000
...,...,...
137,390.00,3844
138,388.75,3677
139,450.00,3820
140,440.00,3280


In [14]:
df_subset_rent = df_subset_rent.groupby('Postcode', as_index=False)['Average'].mean()

df_subset_rent

,Postcode,Average
0,3000,587.50
1,3002,568.75
2,3003,526.25
3,3006,600.00
4,3008,595.00
...,...,...
129,3929,502.50
130,3930,608.25
131,3936,526.25
132,3976,457.50


In [15]:
df_pode

,SA2 Name,Population Density
0,Alfredton,360.4
1,Ballarat,954.0
2,Buninyong,142.0
3,Delacombe,376.7
4,Smythes Creek,40.8
...,...,...
517,Otway,2.6
518,Moyne - East,2.2
519,Moyne - West,5.0
520,Warrnambool - North,364.7


In [23]:
df_pode['SA2 Name'] = df_pode['SA2 Name'].apply(lambda x: f"{x}, Victoria, Australia" if pd.notna(x) else x)

df_pode

,SA2 Name,Population Density
0,"Alfredton, Victoria, Australia",360.4
1,"Ballarat, Victoria, Australia",954.0
2,"Buninyong, Victoria, Australia",142.0
3,"Delacombe, Victoria, Australia",376.7
4,"Smythes Creek, Victoria, Australia",40.8
...,...,...
517,"Otway, Victoria, Australia",2.6
518,"Moyne - East, Victoria, Australia",2.2
519,"Moyne - West, Victoria, Australia",5.0
520,"Warrnambool - North, Victoria, Australia",364.7


## Get the Postcode for Population Density Part

In [24]:
API_KEY = 'Your Keys'
geolocator = GoogleV3(api_key=API_KEY)

def get_postcode(sa2_name):
    try:
        location = geolocator.geocode(sa2_name)
        if location:
            for component in location.raw['address_components']:
                if 'postal_code' in component['types']:
                    return component['long_name']
        return None
    except Exception as e:
        print(f"Error fetching postcode for {sa2_name}: {e}")
        return None
df_pode['Postcode'] = df_pode['SA2 Name'].apply(get_postcode)

df_pode

,SA2 Name,Population Density,Postcode
0,"Alfredton, Victoria, Australia",360.4,3350
1,"Ballarat, Victoria, Australia",954.0,None
2,"Buninyong, Victoria, Australia",142.0,3357
3,"Delacombe, Victoria, Australia",376.7,3356
4,"Smythes Creek, Victoria, Australia",40.8,3351
...,...,...,...
517,"Otway, Victoria, Australia",2.6,3233
518,"Moyne - East, Victoria, Australia",2.2,None
519,"Moyne - West, Victoria, Australia",5.0,None
520,"Warrnambool - North, Victoria, Australia",364.7,3280


In [4]:
df_pode = pd.read_excel('../../ADS ASS2/Data/df_pode.xlsx')

In [5]:
df_pode = df_pode.dropna(subset=['Postcode'])
df_subset_pode = df_pode[['Population Density', 'Postcode']]
df_subset_pode

,Population Density,Postcode
0,360.4,3350.0
2,142.0,3357.0
3,376.7,3356.0
4,40.8,3351.0
5,228.4,3355.0
...,...,...
513,230.9,3250.0
514,2.9,3250.0
517,2.6,3233.0
520,364.7,3280.0


In [6]:
df_subset_pode = df_subset_pode.groupby('Postcode', as_index=False)['Population Density'].mean()

df_subset_pode

,Postcode,Population Density
0,3000.0,24599.233333
1,3002.0,1758.700000
2,3003.0,5289.200000
3,3004.0,0.000000
4,3006.0,16024.800000
...,...,...
314,3977.0,1812.137500
315,3978.0,1436.400000
316,3981.0,27.800000
317,3996.0,38.800000


## By Postcode to combine Population Density and Weekly Rent Price

In [16]:
df_merged = pd.merge(df_subset_pode, df_subset_rent, on='Postcode', how='inner')
df_merged

,Postcode,Population Density,Average
0,3000.0,24599.233333,587.50
1,3002.0,1758.700000,568.75
2,3003.0,5289.200000,526.25
3,3006.0,16024.800000,600.00
4,3008.0,7251.600000,595.00
...,...,...,...
126,3929.0,20.900000,502.50
127,3930.0,813.400000,608.25
128,3936.0,301.300000,526.25
129,3976.0,2136.550000,457.50


In [18]:
file_path = '../../ADS ASS2/Data/final_population_density.xlsx'

df_merged.to_excel(file_path, index=False)

In [17]:
correlation = df_merged['Average'].corr(df_merged['Population Density'])
correlation

0.41235250988759997